In [1]:
from google.colab import drive
drive.mount("/content/drive")
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import (
    roc_auc_score, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score, classification_report
)
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/Multicampus-8/4_머신러닝_딥러닝/프로젝트/'

train_df = pd.read_csv(DATA_PATH + "dataset/train.csv")
test_df = pd.read_csv(DATA_PATH + "dataset/test.csv")
target = "diagnosed_diabetes"
X = train_df.drop(["id", target], axis=1, errors="ignore")
y = train_df[target]

X_train_raw, X_val_raw, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import lightgbm as lgb
import warnings
from tqdm.auto import tqdm

warnings.filterwarnings('ignore')


features = [
    'age','physical_activity_minutes_per_week','diet_score','bmi',
    'systolic_bp','triglycerides','hdl_cholesterol',
    'family_history_diabetes','hypertension_history'
]

X = train_df[features]
y = train_df['diagnosed_diabetes']

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

preprocessor = Pipeline([('scaler', StandardScaler())])


model_configs = [
    {
        "name": "Logistic Regression",
        "model": LogisticRegression(max_iter=2000, class_weight='balanced', random_state=42),
        "params": {"model__C": [0.01,0.1,1,5,10]}
    },
    {
        "name": "Decision Tree",
        "model": DecisionTreeClassifier(class_weight='balanced', random_state=42),
        "params": {"model__max_depth":[3,5,7,10,None],"model__min_samples_leaf":[5,10,20]}
    },
    {
        "name": "XGBoost",
        "model": xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc', use_label_encoder=False, random_state=42),
        "params": {
            "model__n_estimators":[300,500,700],
            "model__learning_rate":[0.01,0.03,0.05],
            "model__max_depth":[3,4,5],
            "model__subsample":[0.8,1.0],
            "model__colsample_bytree":[0.8,1.0]
        }
    },
    {
        "name": "LightGBM",
        "model": lgb.LGBMClassifier(objective='binary', class_weight='balanced', random_state=42),
        "params": {
            "model__n_estimators":[600,800,1000],
            "model__learning_rate":[0.01,0.02,0.03],
            "model__num_leaves":[31,63,127],
            "model__subsample":[0.8,0.9],
            "model__colsample_bytree":[0.8,0.9]
        }
    }
]


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = []

for config in tqdm(model_configs, desc="모델 최적화 진행 중"):
    pipeline = Pipeline([('prep', preprocessor), ('model', config['model'])])

    search = RandomizedSearchCV(
        pipeline,
        param_distributions=config['params'],
        n_iter=10,
        cv=cv,
        scoring='roc_auc',
        n_jobs=-1,
        random_state=42,
        verbose=1
    )

    search.fit(X_train, y_train)

    val_probs = search.predict_proba(X_valid)[:, 1]
    val_auc = roc_auc_score(y_valid, val_probs)

    results.append({
        "Model": config['name'],
        "Val_AUC": round(val_auc,4),
        "Best_CV_AUC": round(search.best_score_,4),
        "Best_Params": search.best_params_
    })

results_df = pd.DataFrame(results).sort_values(by='Val_AUC', ascending=False).reset_index(drop=True)
display(results_df)


best_model_info = max(results, key=lambda x: x['Val_AUC'])
best_model_name = best_model_info['Model']
best_params = best_model_info['Best_Params']

selected_config = next(config for config in model_configs if config["name"] == best_model_name)
final_model_obj = selected_config["model"]

final_pipeline = Pipeline([
    ("prep", preprocessor),
    ("model", final_model_obj)
])
final_pipeline.set_params(**best_params)


X_full = pd.concat([X_train, X_valid], axis=0)
y_full = pd.concat([y_train, y_valid], axis=0)
final_pipeline.fit(X_full, y_full)


test_X = test_df[features]
test_proba = final_pipeline.predict_proba(test_X)[:, 1]

submission = pd.DataFrame({
    "id": test_df["id"] if "id" in test_df.columns else np.arange(len(test_df)),
    "diagnosed_diabetes": test_proba
})
submission.to_csv("submission_best_model.csv", index=False)
display(submission.head())


모델 최적화 진행 중:   0%|          | 0/4 [00:00<?, ?it/s]

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Info] Number of positive: 349046, number of negative: 210954
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.114823 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 994
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


,Model,Val_AUC,Best_CV_AUC,Best_Params
0,LightGBM,0.7246,0.7240,"{'model__subsample': 0.9, 'model__num_leaves':..."
1,XGBoost,0.7213,0.7210,"{'model__subsample': 1.0, 'model__n_estimators..."
2,Decision Tree,0.6940,0.6918,"{'model__min_samples_leaf': 20, 'model__max_de..."
3,Logistic Regression,0.6933,0.6925,{'model__C': 5}


[LightGBM] [Info] Number of positive: 436307, number of negative: 263693
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.069658 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 994
[LightGBM] [Info] Number of data points in the train set: 700000, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


,id,diagnosed_diabetes
0,700000,0.338476
1,700001,0.600298
2,700002,0.637103
3,700003,0.267660
4,700004,0.876392


In [3]:
import time
from datetime import timedelta
import warnings
from tqdm.auto import tqdm # 진행 바 라이브러리

warnings.filterwarnings('ignore', category=UserWarning)

# 1. 교차 검증 및 결과 저장 준비
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = []

# 2. 전체 모델 루프에 tqdm 적용 (전체 진행률 확인)
for config in tqdm(model_configs, desc="전체 모델 최적화 진행 중"):
    print(f"\n [{config['name']}] 랜덤 서치 탐색을 시작합니다...")

    # 시간 측정 시작
    start_time = time.perf_counter()

    pipeline = Pipeline([("prep", preprocessor), ("model", config['model'])])

    # RandomizedSearchCV 설정
    # verbose=1: 간단한 진행 상황 출력 (각 파라미터 조합 시도 시 출력)
    # n_jobs=-1: 병렬 처리 활성화
    search = RandomizedSearchCV(
        pipeline,
        param_distributions=config['params'],
        n_iter=10,
        cv=cv,
        scoring='roc_auc',
        n_jobs=-1,
        random_state=42,
        verbose=1 # <--- 실행 중인 상태를 로그로 보여줍니다.
    )

    # 모델 학습 (실제 탐색 수행)
    search.fit(X_train, y_train)

    # 시간 측정 종료 및 변환
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    elapsed_hms = str(timedelta(seconds=int(elapsed_time)))

    # 검증셋 평가
    val_probs = search.predict_proba(X_valid)[:, 1]
    val_auc = roc_auc_score(y_valid, val_probs)
    val_pred = search.predict(X_valid)

    # 상세 지표 계산
    tn, fp, fn, tp = confusion_matrix(y_val, val_pred).ravel()

    print(f"[{config['name']}] 탐색 완료!")
    print(f"소요 시간: {elapsed_hms} | Best CV AUC: {search.best_score_:.4f} | Val AUC: {val_auc:.4f}")

    # 결과 저장
    results.append({
        "Model": config['name'],
        "AUC": roc_auc_score(y_val, val_probs),
        "Accuracy": accuracy_score(y_val, val_pred),
        "Precision": precision_score(y_val, val_pred),
        "Recall": recall_score(y_val, val_pred),
        "F1": f1_score(y_val, val_pred),
        "TP": tp, "TN": tn, "FP": fp, "FN": fn,
        "Best_CV_AUC": search.best_score_,
        "Val_AUC": val_auc,
        "Time": elapsed_hms,
        "Best_Params": search.best_params_
    })

전체 모델 최적화 진행 중:   0%|          | 0/4 [00:00<?, ?it/s]


 [Logistic Regression] 랜덤 서치 탐색을 시작합니다...
Fitting 5 folds for each of 5 candidates, totalling 25 fits
[Logistic Regression] 탐색 완료!
소요 시간: 0:00:13 | Best CV AUC: 0.6925 | Val AUC: 0.6933

 [Decision Tree] 랜덤 서치 탐색을 시작합니다...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[Decision Tree] 탐색 완료!
소요 시간: 0:01:39 | Best CV AUC: 0.6918 | Val AUC: 0.6940

 [XGBoost] 랜덤 서치 탐색을 시작합니다...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[XGBoost] 탐색 완료!
소요 시간: 0:13:21 | Best CV AUC: 0.7210 | Val AUC: 0.7213

 [LightGBM] 랜덤 서치 탐색을 시작합니다...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Info] Number of positive: 349046, number of negative: 210954
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.064395 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 994
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 9
[LightGBM] [Info] [

In [8]:
import pandas as pd
results_df = pd.DataFrame(results)
results_df

,Model,AUC,Accuracy,Precision,Recall,F1,TP,TN,FP,FN,Best_CV_AUC,Val_AUC,Time,Best_Params
0,Logistic Regression,0.693290,0.625829,0.754187,0.592945,0.663916,51741,35875,16864,35520,0.692541,0.693290,0:00:13,{'model__C': 5}
1,Decision Tree,0.693995,0.630200,0.752250,0.606422,0.671510,52917,35311,17428,34344,0.691762,0.693995,0:01:39,"{'model__min_samples_leaf': 20, 'model__max_de..."
2,XGBoost,0.721297,0.680900,0.702990,0.845085,0.767517,73743,21583,31156,13518,0.721047,0.721297,0:13:21,"{'model__subsample': 1.0, 'model__n_estimators..."
3,LightGBM,0.724612,0.653879,0.772623,0.630133,0.694141,54986,36557,16182,32275,0.723985,0.724612,0:39:40,"{'model__subsample': 0.9, 'model__num_leaves':..."


In [9]:
display(results_df[["Model", "Best_CV_AUC", "Val_AUC", "Time"]])

,Model,Best_CV_AUC,Val_AUC,Time
0,Logistic Regression,0.692541,0.693290,0:00:13
1,Decision Tree,0.691762,0.693995,0:01:39
2,XGBoost,0.721047,0.721297,0:13:21
3,LightGBM,0.723985,0.724612,0:39:40


In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 성능(AUC)과 시간(초 단위) 비교 차트
results_df['Seconds'] = results_df['Time'].apply(lambda x: sum(int(a) * 60**i for i, a in enumerate(reversed(x.split(':')))))

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=results_df['Model'], y=results_df['Val_AUC'], name="Validation AUC"), secondary_y=False)
fig.add_trace(go.Scatter(x=results_df['Model'], y=results_df['Seconds'], name="Time (seconds)", mode='lines+markers'), secondary_y=True)

fig.update_layout(title_text="모델별 성능 및 최적화 소요 시간 비교")
fig.update_yaxes(title_text="AUC Score", secondary_y=False)
fig.update_yaxes(title_text="Time (s)", secondary_y=True)
fig.show()

In [11]:
# 1. 랜덤 서치 결과 중 가장 성능(Val_AUC)이 좋은 모델 정보 가져오기
best_model_info = max(results, key=lambda x: x['Val_AUC'])
best_model_name = best_model_info['Model']
best_params = best_model_info['Best_Params']

print(f"🏆 최종 선택된 모델: {best_model_name}")
print(f"⚙️ 최적 하이퍼파라미터: {best_params}")

# 2. 최적 모델 객체 찾기 및 파라미터 적용
# model_configs 리스트에서 이름이 같은 모델 설정을 찾습니다.
selected_config = next(config for config in model_configs if config["name"] == best_model_name)
final_model_obj = selected_config["model"]

# 파생된 파이프라인 생성 (전처리기 + 선택된 모델)
final_pipeline = Pipeline([
    ("prep", preprocessor),
    ("model", final_model_obj)
])

# RandomizedSearchCV에서 찾은 'best_params'를 파이프라인에 직접 주입합니다.
final_pipeline.set_params(**best_params)

# 3. 전체 데이터로 재학습 (Train + Validation 합쳐서 학습하면 성능이 더 좋아집니다)
# 강의용이라면 간단하게 X_train_raw만 사용해도 무방합니다.
print("최적 파라미터로 모델 재학습 중...")
final_pipeline.fit(X_train, y_train)

# 4. 테스트 데이터 예측
test_X = test_df[features]
test_proba = final_pipeline.predict_proba(test_X)[:, 1]

# 5. 제출 데이터프레임 구성
submission = pd.DataFrame({
    "id": test_df["id"] if "id" in test_df.columns else np.arange(len(test_df)),
    "diagnosed_diabetes": test_proba
})

# 6. 파일 저장 및 확인
file_name = f"submission_best_{best_model_name.lower().replace(' ', '_')}.csv"
save_path = DATA_PATH + "output/" + file_name
submission.to_csv(save_path, index=False)
print(f"제출 파일 저장 완료: {save_path}")
display(submission.head())

🏆 최종 선택된 모델: LightGBM
⚙️ 최적 하이퍼파라미터: {'model__subsample': 0.9, 'model__num_leaves': 63, 'model__n_estimators': 800, 'model__learning_rate': 0.03, 'model__colsample_bytree': 0.8}
최적 파라미터로 모델 재학습 중...
[LightGBM] [Info] Number of positive: 349046, number of negative: 210954
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.055406 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 994
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
제출 파일 저장 완료: /content/drive/MyDrive/Colab Notebooks/Multicampus-8/4_머신러닝_딥러닝/프로젝트/output/submission_best_lightgbm.csv


,id,diagnosed_diabetes
0,700000,0.343998
1,700001,0.620765
2,700002,0.651108
3,700003,0.266749
4,700004,0.884840


In [12]:
# 5. 파일 다운로드 (Colab)
from google.colab import files
files.download(save_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>